In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math
from collections import Counter

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

n_children = 1000000 # n children to give
n_gift_type = 1000 # n types of gifts available
n_gift_quantity = 1000 # each type of gifts are limited to this quantity
n_gift_pref = 100 # number of gifts a child ranks
n_child_pref = 1000 # number of children a gift ranks
twins = math.ceil(0.04 * n_children / 2.) * 2    # 4% of all population, rounded to the closest number
triplets = math.ceil(0.005 * n_children / 3.) * 3    # 0.5% of all population, rounded to the closest number
ratio_gift_happiness = 2
ratio_child_happiness = 2



gift_pref = pd.read_csv('../input/child_wishlist_v2.csv',header=None).drop(0, 1).values
child_pref = pd.read_csv('../input/gift_goodkids_v2.csv',header=None).drop(0, 1).values


def lcm(a, b):
    """Compute the lowest common multiple of a and b"""
    # in case of large numbers, using floor division
    return a * b // math.gcd(a, b)

def avg_normalized_happiness(pred, child_pref, gift_pref):
    
    # check if number of each gift exceeds n_gift_quantity
    gift_counts = Counter(elem[1] for elem in pred)
    for count in gift_counts.values():
        assert count <= n_gift_quantity
                
    # check if triplets have the same gift
    for t1 in np.arange(0,triplets,3):
        triplet1 = pred[t1]
        triplet2 = pred[t1+1]
        triplet3 = pred[t1+2]
        # print(t1, triplet1, triplet2, triplet3)
        assert triplet1[1] == triplet1[1] and triplet2[1] == triplet3[1]
                
    # check if twins have the same gift
    for t1 in np.arange(triplets,triplets+twins,2):
        twin1 = pred[t1]
        twin2 = pred[t1+1]
        # print(t1)
        assert twin1[1] == twin2[1]

    max_child_happiness = n_gift_pref * ratio_child_happiness
    max_gift_happiness = n_child_pref * ratio_gift_happiness
    total_child_happiness = 0
    total_gift_happiness = np.zeros(n_gift_type)
    
    for row in pred:
        child_id = row[0]
        gift_id = row[1]
        
        # check if child_id and gift_id exist
        assert child_id < n_children
        assert gift_id < n_gift_type
        assert child_id >= 0 
        assert gift_id >= 0
        child_happiness = (n_gift_pref - np.where(gift_pref[child_id]==gift_id)[0]) * ratio_child_happiness
        if not child_happiness:
            child_happiness = -1

        gift_happiness = ( n_child_pref - np.where(child_pref[gift_id]==child_id)[0]) * ratio_gift_happiness
        if not gift_happiness:
            gift_happiness = -1

        total_child_happiness += child_happiness
        total_gift_happiness[gift_id] += gift_happiness
    
    print('normalized child happiness=',float(total_child_happiness)/(float(n_children)*float(max_child_happiness)) , \
        ', normalized gift happiness',np.mean(total_gift_happiness) / float(max_gift_happiness*n_gift_quantity))

    # to avoid float rounding error
    # find common denominator
    # NOTE: I used this code to experiment different parameters, so it was necessary to get the multiplier
    # Note: You should hard-code the multipler to speed up, now that the parameters are finalized
    denominator1 = n_children*max_child_happiness
    denominator2 = n_gift_quantity*max_gift_happiness*n_gift_type
    common_denom = lcm(denominator1, denominator2)
    multiplier = common_denom / denominator1

    # # usually denom1 > demon2
    return float(math.pow(total_child_happiness*multiplier,3) + math.pow(np.sum(total_gift_happiness),3)) / float(math.pow(common_denom,3))
    # return math.pow(float(total_child_happiness)/(float(n_children)*float(max_child_happiness)),2) + math.pow(np.mean(total_gift_happiness) / float(max_gift_happiness*n_gift_quantity),2)

random_sub = pd.read_csv('../input/sample_submission_random_v2.csv').values.tolist()
print(avg_normalized_happiness(random_sub, child_pref, gift_pref))

child_wishlist.csv
child_wishlist_v2.csv
gift_goodkids.csv
gift_goodkids_v2.csv
sample_submission_random.csv
sample_submission_random_v2.csv
santa-gift-matching-publicleaderboard.csv

normalized child happiness= 0.04621203 , normalized gift happiness -4.59355e-05
9.868817990273061e-05


In [2]:
gift_pref.shape, child_pref.shape

((1000000, 100), (1000, 1000))

In [3]:
class Child(object):
    
    def __init__(self, idx, prefer):
        
        self.idx = idx
        self.prefer_dict = dict()
        
        for i in range(prefer.shape[0]):
            self.prefer_dict[prefer[i]] = [12*(prefer.shape[0] - i), -6]
    
    
    def add_gifts_prefer(self, giftid, score):
        
        if giftid in self.prefer_dict.keys():
            self.prefer_dict[giftid][1] = 6*score
        else:
            self.prefer_dict[giftid] = [-6, 6*score]
        
        return None
        
    
    def happiness(self, giftid):
        
        return self.prefer_dict.get(giftid, [-6, -6])
    
class Child_twin(object):
    
    def __init__(self, idx, prefer1, prefer2):
        
        self.idx = idx
        self.prefer_dict = dict()
        
        for p in list(set(list(prefer1) + list(prefer2))):
            score = 0
            if p in list(prefer1):
                score += 2*(100 - list(prefer1).index(p))
            else:
                score -= 1
            if p in list(prefer2):
                score += 2*(100 - list(prefer2).index(p))
            else:
                score -= 1
            self.prefer_dict[p] = [3*score, -6]
    
    
    def add_gifts_prefer(self, giftid, score):
        
        if giftid in self.prefer_dict.keys():
            self.prefer_dict[giftid][1] = 3*score
        else:
            self.prefer_dict[giftid] = [-6, 3*score]
        
        return None
        
    
    def happiness(self, giftid):
        
        return self.prefer_dict.get(giftid, [-6, -6])
    
class Child_triplet(object):
    
    def __init__(self, idx, prefer1, prefer2, prefer3):
        
        self.idx = idx
        self.prefer_dict = dict()
        
        for p in list(set(list(prefer1) + list(prefer2) + list(prefer3))):
            score = 0
            if p in list(prefer1):
                score += 2*(100 - list(prefer1).index(p))
            else:
                score -= 1
            if p in list(prefer2):
                score += 2*(100 - list(prefer2).index(p))
            else:
                score -= 1
            if p in list(prefer3):
                score += 2*(100 - list(prefer3).index(p))
            else:
                score -= 1
            self.prefer_dict[p] = [2*score, -6]
    
    
    def add_gifts_prefer(self, giftid, score):
        
        if giftid in self.prefer_dict.keys():
            self.prefer_dict[giftid][1] = 2*score
        else:
            self.prefer_dict[giftid] = [-6, 2*score]
        
        return None
        
    
    def happiness(self, giftid):
        
        return self.prefer_dict.get(giftid, [-6, -6])
    
    
Children = []
for i in range(0, 5001, 3):
    Children.append(Child_triplet(i, gift_pref[i], gift_pref[i+1], gift_pref[i+2]))
    Children.append(Child_triplet(i+1, gift_pref[i], gift_pref[i+1], gift_pref[i+2]))
    Children.append(Child_triplet(i+2, gift_pref[i], gift_pref[i+1], gift_pref[i+2]))
for i in range(5001, 45001, 2):
    Children.append(Child_twin(i, gift_pref[i], gift_pref[i+1]))
    Children.append(Child_twin(i+1, gift_pref[i], gift_pref[i+1]))
Children = Children + [Child(i, gift_pref[i]) for i in range(45001, 1000000)]

for j in range(1000):
    cf = child_pref[j]
    done_list = []
    for i in range(cf.shape[0]):
        if cf[i] <= 5000 and cf[i] not in done_list:
            if cf[i] % 3 == 0:
                cid1 = cf[i]
                cid2 = cf[i] + 1
                cid3 = cf[i] + 2
                done_list.append(cid2)
                done_list.append(cid3)
            elif cf[i] % 3 == 1:
                cid1 = cf[i] - 1
                cid2 = cf[i]
                cid3 = cf[i] + 1
                done_list.append(cid1)
                done_list.append(cid3)
            else:
                cid1 = cf[i] - 2
                cid2 = cf[i] - 1
                cid3 = cf[i]
                done_list.append(cid1)
                done_list.append(cid2)
            if cid1 in list(cf):
                score_ = 2*(cf.shape[0] - list(cf).index(cid1))
            else:
                score_ = -1
            if cid2 in list(cf):
                score_ += 2*(cf.shape[0] - list(cf).index(cid2))
            else:
                score_ += -1
            if cid3 in list(cf):
                score_ += 2*(cf.shape[0] - list(cf).index(cid3))
            else:
                score_ += -1
            Children[cid1].add_gifts_prefer(j, score_)
            Children[cid2].add_gifts_prefer(j, score_)
            Children[cid3].add_gifts_prefer(j, score_)
        elif cf[i] <= 45000 and cf[i] not in done_list:
            if cf[i] % 2 == 0:
                cid1 = cf[i]
                cid2 = cf[i] + 1
                done_list.append(cid2)
            else:
                cid1 = cf[i] - 1
                cid2 = cf[i]
                done_list.append(cid1)
            if cid1 in list(cf):
                score_ = 2*(cf.shape[0] - list(cf).index(cid1))
            else:
                score_ = -1
            if cid2 in list(cf):
                score_ += 2*(cf.shape[0] - list(cf).index(cid2))
            else:
                score_ += -1
            Children[cid1].add_gifts_prefer(j, score_)
            Children[cid2].add_gifts_prefer(j, score_)
        elif cf[i] > 45000:
            Children[cf[i]].add_gifts_prefer(j, 2*(cf.shape[0] - i))

In [4]:
subm = pd.read_csv('../submit/subm_hrd_new_3.csv')

In [5]:
CHILD_HAPPINESS = sum([Children[i].happiness(subm.GiftId.values[i])[0] for i in range(1000000)])*10
SANTA_HAPPINESS = sum([Children[i].happiness(subm.GiftId.values[i])[1] for i in range(1000000)])
OBJ = CHILD_HAPPINESS**3 + SANTA_HAPPINESS**3
print(CHILD_HAPPINESS, SANTA_HAPPINESS, OBJ)
print(OBJ / (12000000000**3))

11739570540 1681860 1617918456177222292748812320000
0.936295402880337


In [6]:
gift_count_trips = [0 for _ in range(1000)]
gift_count_twins = [0 for _ in range(1000)]
gift_count_sings = [0 for _ in range(1000)]

for i in range(0, 5001, 3):
    j = subm.GiftId.values[i]
    gift_count_trips[j] += 1
for i in range(5001, 45001, 2):
    j = subm.GiftId.values[i]
    gift_count_twins[j] += 1
for i in range(45001, 1000000):
    j = subm.GiftId.values[i]
    gift_count_sings[j] += 1

In [9]:
from ortools.graph import pywrapgraph

W_CHILD = 1
W_GIFTS = 0

In [7]:
5001 / 3 + 40000 / 2 + (1000000-45001)

976666.0

In [10]:
min_cost_flow_mod = pywrapgraph.SimpleMinCostFlow()

start_nodes = []
end_nodes = []
capacities = []
unit_costs = []


# triplets
for i in range(0, 5001, 3):
    for g in Children[i].prefer_dict.keys():
        start_nodes.append(1002000+g)
        end_nodes.append(i)
        capacities.append(1)
        unit_costs.append(3*(-W_CHILD*(Children[i].prefer_dict[g][0] + 6)-W_GIFTS*(Children[i].prefer_dict[g][1] + 6)))
        
# triplets
for i in range(5001, 45001, 2):
    for g in Children[i].prefer_dict.keys():
        start_nodes.append(1001000+g)
        end_nodes.append(i)
        capacities.append(1)
        unit_costs.append(2*(-W_CHILD*(Children[i].prefer_dict[g][0] + 6)-W_GIFTS*(Children[i].prefer_dict[g][1] + 6)))
        
# other children
for i in range(45001, 1000000):
    
    for g in Children[i].prefer_dict.keys():
        start_nodes.append(1000000+g)
        end_nodes.append(i)
        capacities.append(1)
        unit_costs.append(-W_CHILD*(Children[i].prefer_dict[g][0] + 6)-W_GIFTS*(Children[i].prefer_dict[g][1] + 6))

# add Arc
# gift -> children
for i in range(len(start_nodes)):
    min_cost_flow_mod.AddArcWithCapacityAndUnitCost(
        int(start_nodes[i]), int(end_nodes[i]), int(capacities[i]), int(unit_costs[i])
    )
    
# children -> 1003000 : collection
for i in range(0, 5001, 3):
    min_cost_flow_mod.AddArcWithCapacityAndUnitCost(
        int(i), int(1003000), int(1), int(0)
    )
for i in range(5001, 45001, 2):
    min_cost_flow_mod.AddArcWithCapacityAndUnitCost(
        int(i), int(1003000), int(1), int(0)
    )
for i in range(45001, 1000000):
    min_cost_flow_mod.AddArcWithCapacityAndUnitCost(
        int(i), int(1003000), int(1), int(0)
    )
    
# gift -> 1003001 : dust_gift
for i in range(1000):
    min_cost_flow_mod.AddArcWithCapacityAndUnitCost(
        int(1002000+i), int(1003001), int(1000), int(0)
    )
    min_cost_flow_mod.AddArcWithCapacityAndUnitCost(
        int(1001000+i), int(1003001), int(1000), int(0)
    )
    min_cost_flow_mod.AddArcWithCapacityAndUnitCost(
        int(1000000+i), int(1003001), int(1000), int(0)
    )
    
# 1003001 -> 1003000 : dust_path
min_cost_flow_mod.AddArcWithCapacityAndUnitCost(
    int(1003001), int(1003000), int(1000000), int(0)
)


# add Supply
for i in range(1000):
    min_cost_flow_mod.SetNodeSupply(int(1002000+i), int(gift_count_trips[i]))
    min_cost_flow_mod.SetNodeSupply(int(1001000+i), int(gift_count_twins[i]))
    min_cost_flow_mod.SetNodeSupply(int(1000000+i), int(gift_count_sings[i]))

# children
for i in range(0, 5001, 3):
    min_cost_flow_mod.SetNodeSupply(int(i), int(0))
for i in range(5001, 45001, 2):
    min_cost_flow_mod.SetNodeSupply(int(i), int(0))
for i in range(45001, 1000000):
    min_cost_flow_mod.SetNodeSupply(int(i), int(0))

min_cost_flow_mod.SetNodeSupply(int(1003001), int(0)) 
min_cost_flow_mod.SetNodeSupply(int(1003000), int(-976666)) 



In [11]:
min_cost_flow_mod.Solve()

1

In [12]:
assignment = [-1]*1000000

for i in range(min_cost_flow_mod.NumArcs()):
    if min_cost_flow_mod.Flow(i) != 0 and min_cost_flow_mod.Head(i) < 1000000:
        c = min_cost_flow_mod.Head(i)
        g = min_cost_flow_mod.Tail(i)
        f = min_cost_flow_mod.Flow(i)

        if c >= 45001:
            assignment[c] = g - 1000000

        elif c >= 5001:
            assignment[c]   = g - 1001000
            assignment[c+1] = g - 1001000
        else:
            assignment[c]   = g - 1002000
            assignment[c+1] = g - 1002000
            assignment[c+2] = g - 1002000
                
CHILD_HAPPINESS = sum([Children[i].happiness(assignment[i])[0] for i in range(1000000)])*10
SANTA_HAPPINESS = sum([Children[i].happiness(assignment[i])[1] for i in range(1000000)])
OBJ = CHILD_HAPPINESS**3 + SANTA_HAPPINESS**3
print(W_CHILD, W_GIFTS, CHILD_HAPPINESS, SANTA_HAPPINESS, OBJ)
print(OBJ / (12000000000**3))

1 0 11739594660 362418 1617928428668493164308005994632
0.9363011739979705


wata: 0.9362995374

In [14]:
unassigned_v = [0, 0, 0]
unassigned_v_ = [0, 0]
for i in range(5001):
    if assignment[i] == -1:
        unassigned_v[0] += 1
for i in range(5001, 45001):
    if assignment[i] == -1:
        unassigned_v[1] += 1
for i in range(45001, 1000000):
    if assignment[i] == -1:
        unassigned_v[2] += 1 
for i in range(0, 5001, 3):
    if assignment[i] == -1:
        unassigned_v_[0] += 1
for i in range(5001, 45001, 2):
    if assignment[i] == -1:
        unassigned_v_[1] += 1
        
unassigned_v, unassigned_v_

([2760, 954, 0], [920, 477])

In [13]:
Gifts_left = [1000 for _ in range(1000)]
for i in range(1000000):
    if assignment[i] != -1:
        Gifts_left[assignment[i]] -= 1
for i in range(1000):
    if Gifts_left[i] != 0:
        print(i, Gifts_left[i])

118 631
240 184
272 148
320 497
389 151
494 981
671 709
998 413


In [16]:
assignment_save = assignment.copy()

In [15]:
gifts_for_trips = [631, 184, 148, 497, 151, 981, 709, 413]
gifts_for_twins = [4, 4, 4, 2, 4, 930, 4, 2]
for i in range(8):
    gifts_for_trips[i] -= gifts_for_twins[i]

In [17]:
gifts_left_list = []
for i in range(gifts_for_trips[0]):
    gifts_left_list.append(118)
for i in range(gifts_for_trips[1]):
    gifts_left_list.append(240)
for i in range(gifts_for_trips[2]):
    gifts_left_list.append(272)
for i in range(gifts_for_trips[3]):
    gifts_left_list.append(320)
for i in range(gifts_for_trips[4]):
    gifts_left_list.append(389)
for i in range(gifts_for_trips[5]):
    gifts_left_list.append(494)
for i in range(gifts_for_trips[6]):
    gifts_left_list.append(671)
for i in range(gifts_for_trips[7]):
    gifts_left_list.append(998)
    
for i in range(gifts_for_twins[0]):
    gifts_left_list.append(118)
for i in range(gifts_for_twins[1]):
    gifts_left_list.append(240)
for i in range(gifts_for_twins[2]):
    gifts_left_list.append(272)
for i in range(gifts_for_twins[3]):
    gifts_left_list.append(320)
for i in range(gifts_for_twins[4]):
    gifts_left_list.append(389)
for i in range(gifts_for_twins[5]):
    gifts_left_list.append(494)
for i in range(gifts_for_twins[6]):
    gifts_left_list.append(671)
for i in range(gifts_for_twins[7]):
    gifts_left_list.append(998)

In [18]:
ind = 0
for i in range(1000000):
    if assignment[i] == -1:
        assignment[i] = gifts_left_list[ind]
        ind += 1

In [19]:
out = open('../submit/subm_hrd_new_4.csv', 'w')
out.write('ChildId,GiftId\n')
for i in range(1000000):
    out.write(str(i) + ',' + str(assignment[i]) + '\n')
out.close()